In [1]:
import networkx as nx
import dwave_networkx as dnx
import minorminer
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import random
from dwave.system import DWaveSampler
import sys
from minorminer import find_embedding
from dwave.embedding.zephyr import find_biclique_embedding
from model.rbm.rbm_two_partite import RBM_TwoPartite
import torch
import torch
from hydra.utils import instantiate
from hydra import initialize, compose
import hydra
from omegaconf import OmegaConf
import os
from scripts.run import setup_model, load_model_instance
from utils.rbm_plots import rbm_to_networkx, plot_pruning_analysis
import dwave_networkx as dnx
import dwave.inspector





[22:14:10.794] INFO   CaloQuVAE                                         Loading configuration.


In [9]:
solver_name = "Advantage2_system1.9"
TARGET_SOLVER = solver_name
try:
    target_sampler = DWaveSampler(solver=TARGET_SOLVER)
except Exception as e:
    print(f"Error initializing sampler: {e}")
    print("Please ensure you have D-Wave credentials configured.")

# Use the sampler's graph directly
working_graph = target_sampler.to_networkx_graph() 

try:
    left_chains, right_chains = find_biclique_embedding(
        75, 75, target_graph=working_graph
    )
except Exception as e:
    print(f"Error during find_biclique_embedding: {e}")
    print("This often means the graph is too large for the QPU.")


[22:15:45.931] INFO   dwave.cloud.config.models                         Invalid solver JSON, parsing as string identity: 'Advantage2_system1.9'
[22:15:46.211] INFO   dwave.cloud.client.base                           Fetching definition of a solver with name='Advantage2_system1.9'
[22:15:46.234] INFO   dwave.cloud.client.base                           Received solver data for 1 solver(s).
[22:15:46.235] INFO   dwave.cloud.client.base                           Adding solver StructuredSolver(name='Advantage2_system1.9', graph_id='014a2885f7')


In [55]:
hydra.core.global_hydra.GlobalHydra.instance().clear()
initialize(version_base=None, config_path="config")
cfg=compose(config_name="config.yaml")
new_model = False
if new_model:
    self = setup_model(config)
else:
    config = OmegaConf.load(cfg.config_path)
    config.gpu_list = cfg.gpu_list
    config.load_state = cfg.load_state
    self = setup_model(config)
    self._model_creator.load_state(config.run_path, self.device)

# wandb.init(tags = [cfg.data.dataset_name], project=cfg.wandb.project, entity=cfg.wandb.entity, config=OmegaConf.to_container(cfg, resolve=True), mode='disabled')
dummy_data = torch.zeros(1, config.rbm.latent_nodes_per_p).to(self.device)
CHECKPOINT_FILE = "/home/leozhu/CaloQuVAE/wandb-outputs/run_2025-11-09_19-19-34_RBM_TwoPartite/training_checkpoint.h5"
rbm = RBM_TwoPartite(config, data=dummy_data)

try:
    # Load the latest epoch (epoch=None)
    loaded_epoch = rbm.load_checkpoint(CHECKPOINT_FILE, epoch=None) 
    print(f"Successfully loaded checkpoint from epoch {loaded_epoch}.")
except Exception as e:
    print(f"Error loading checkpoint: {e}")

weights = rbm.params["weight_matrix"]
vbias = rbm.params["vbias"]
print(f"  Weight matrix shape: {weights.shape}")
print(f"  Visible bias mean: {vbias.mean().item():.4f}")




In [50]:
G_target = dnx.zephyr_graph(12)   # choose m that gives enough qubits (increase m if needed)
print(f"Target zephyr_graph (m=12): {G_target.number_of_nodes()} nodes, {G_target.number_of_edges()} edges")


In [56]:
print("--- Running Pruning Analysis Plot ---")
# Let's check tolerances up to 0.05
plot_pruning_analysis(rbm, max_tolerance=0.2, num_steps=100)

In [57]:
TOLERANCE_CUTOFF = 0.20
print(f"\n--- Converting to NetworkX Graph (tolerance={TOLERANCE_CUTOFF}) ---")
G = rbm_to_networkx(rbm, tolerance=TOLERANCE_CUTOFF)

# You can now analyze this graph
print(f"Graph has {G.number_of_nodes()} nodes and {G.number_of_edges()} edges.")
# For example, find the visible node with the most connections
vis_nodes = {n for n, d in G.nodes(data=True) if d['bipartite'] == 0}
degrees = G.degree(vis_nodes)
if degrees:
    max_deg_node, max_deg = max(degrees, key=lambda item: item[1])
    print(f"Visible node {max_deg_node} has the highest degree: {max_deg}")

In [58]:
TARGET_SOLVER = "Advantage2_system1.7" 
sampler = DWaveSampler(solver=TARGET_SOLVER)
target_edgelist = sampler.structure.edgelist

print(f" -> Fetched working graph with {len(sampler.structure.nodelist)} active qubits.")
print(f" -> Fetched working graph with {len(target_edgelist)} active couplers.\n")

source_edgelist = list(G.edges())
target_edgelist = sampler.structure.edgelist

embedding = find_embedding(source_edgelist, target_edgelist)

if not embedding:
    print("Result: FAILED")
    print(f"minorminer.find_embedding returned an empty dictionary.")
    print(f"graph could NOT be embedded onto {TARGET_SOLVER}.")
else:
    print("Result: SUCCESS")
    print(f"graph WAS successfully embedded!")
    print("\nEmbedding Details:")


In [62]:
G_target = sampler.to_networkx_graph()
dwave.inspector.show(
        G,      # Your problem graph
        embedding,     # Your embedding dictionary
        G_target       # The hardware graph
    )

In [51]:
def run_embedding(rbm_size, solver_name):
    """
    Performs the biclique embedding on the target QPU.
    Returns the sampler, the working graph, and the chains.
    """
    print(f"--- 1. Running Embedding for K_{rbm_size},{rbm_size} on {solver_name} ---")
    TARGET_SOLVER = solver_name
    try:
        target_sampler = DWaveSampler(solver=TARGET_SOLVER)
    except Exception as e:
        print(f"Error initializing sampler: {e}")
        print("Please ensure you have D-Wave credentials configured.")
        return None, None, None, None, None

    # Use the sampler's graph directly
    working_graph = target_sampler.to_networkx_graph() 
    
    if not working_graph:
        print("Could not fetch working graph from sampler.")
        return target_sampler, None, None, None, None

    print(f"Successfully fetched QPU graph with {len(working_graph.nodes)} nodes.")

    try:
        left_chains, right_chains = find_biclique_embedding(
            rbm_size, rbm_size, target_graph=working_graph
        )
    except Exception as e:
        print(f"Error during find_biclique_embedding: {e}")
        print("This often means the graph is too large for the QPU.")
        return target_sampler, working_graph, None, None, None

    all_chains = list(left_chains.values()) + list(right_chains.values())
    max_len = max(len(chain) for chain in all_chains)

    # This is the set of ALL qubits used in the 76x76 embedding
    qubits_used = set()
    for chain in all_chains:
        qubits_used.update(chain)

    print(f" -> Max chain length: {max_len}")
    print(f" -> Total qubits used: {len(qubits_used)}")
    return target_sampler, working_graph, qubits_used, left_chains, right_chains

def build_neighbor_sets(target_sampler, target_logical_nodes, qubits_used):
    """
    Builds the 76 "available neighbor" sets, V_i.
    V_i = {all *available* qubits adjacent to logical node i}
    """
    print(f"\n--- 2. Building {len(target_logical_nodes)} Neighbor Sets ---")
    
    # 1. Get all qubits available on the chip
    all_physical_qubits = set(target_sampler.nodelist)
    qubits_avail = all_physical_qubits - qubits_used
    print(f"Total available qubits: {len(qubits_avail)}")

    # 2. Get the adjacency property of the sampler
    qpu_adjacency = target_sampler.adjacency
    
    neighbor_sets = []
    min_adj_size = float('inf')
    
    # 3. Iterate over each of the 76 logical nodes in Side A
    for i, chain in enumerate(target_logical_nodes):
        
        # This set will hold all *available* neighbors for this one logical node
        chain_available_neighbors = set()
        
        # 4. Check neighbors for every physical qubit in the chain
        for q_in_chain in chain:
            for neighbor in qpu_adjacency[q_in_chain]:
                # 5. If the neighbor is in the available set, add it
                if neighbor in qubits_avail:
                    chain_available_neighbors.add(neighbor)
        
        neighbor_sets.append(chain_available_neighbors)
        
        if len(chain_available_neighbors) < min_adj_size:
            min_adj_size = len(chain_available_neighbors)
            
    print(f"All 76 neighbor sets built.")
    print(f"The 'bottleneck' (min neighbors) is: {min_adj_size}")
    print(f"This is the *absolute upper bound* on the number of nodes.")
    
    # This is V_all, the total pool of qubits we can *ever* use
    all_available_neighbors = set().union(*neighbor_sets)
    
    return neighbor_sets, all_available_neighbors

def find_max_disjoint_hitting_sets_heuristic(neighbor_sets, all_available_neighbors):
    """
    Heuristic algorithm to find the maximum number of disjoint hitting sets.
    
    Inputs:
    - neighbor_sets (list of sets): The 76 sets V_i.
    - all_available_neighbors (set): The pool of all qubits we can use.
    
    Returns:
    - N_nodes (int): The estimated max number of conditioning nodes.
    - all_found_nodes (list of sets): The physical qubit sets for each node.
    """
    print(f"\n--- 3. Running Greedy Heuristic for Max Disjoint Hitting Sets ---")
    
    N_nodes_found = 0
    all_found_nodes = []
    
    # Make a copy so we can safely modify it
    qubit_pool = all_available_neighbors.copy()
    num_logical_nodes = len(neighbor_sets)
    
    # --- Outer Greedy Loop ---
    # Try to build nodes one by one
    while True:
        
        # --- Inner Greedy Loop ---
        # Try to build *one* valid conditioning node (a hitting set)
        current_hitting_set = set()
        
        # Indices of the logical nodes (0 to 75) we still need to hit
        sets_to_hit_indices = set(range(num_logical_nodes))
        
        # We need a pool of qubits *available for this node*
        # This pool will shrink as we build the current_hitting_set
        current_qubit_pool = qubit_pool.copy()
        
        while sets_to_hit_indices:
            # 1. Find the "best" qubit to add
            best_qubit = None
            max_hits = -1
            
            # This is the greedy "Set Cover" part:
            # Check every available qubit...
            for q in current_qubit_pool:
                current_hits = 0
                # ...to see how many *un-hit* logical nodes it hits
                for i in sets_to_hit_indices:
                    if q in neighbor_sets[i]:
                        current_hits += 1
                
                if current_hits > max_hits:
                    max_hits = current_hits
                    best_qubit = q
            
            # 2. Check if we failed
            if max_hits == 0:
                # We failed to build a complete hitting set
                # The remaining qubits in current_qubit_pool
                # cannot hit the remaining sets_to_hit_indices.
                # Break the inner loop (this node fails)
                break
                
            # 3. Add the best qubit to our node
            current_hitting_set.add(best_qubit)
            
            # 4. Remove it from the pool for *this node*
            current_qubit_pool.remove(best_qubit)
            
            # 5. Update the list of nodes we still need to hit
            indices_hit = set()
            for i in sets_to_hit_indices:
                if best_qubit in neighbor_sets[i]:
                    indices_hit.add(i)
            
            sets_to_hit_indices.difference_update(indices_hit)
            
        # --- End of Inner Loop ---
        
        if not sets_to_hit_indices:
            # SUCCESS! We hit all 76 nodes.
            N_nodes_found += 1
            all_found_nodes.append(current_hitting_set)
            
            # Now, permanently remove these qubits from the *global* pool
            qubit_pool.difference_update(current_hitting_set)
            
            # print(f"  -> Found conditioning node {N_nodes_found} (size {len(current_hitting_set)})")
        else:
            # FAILURE. We couldn't build a new node.
            # The remaining qubits in qubit_pool are not sufficient.
            # Break the outer loop
            break
            
    # --- End of Outer Loop ---
    
    return N_nodes_found, all_found_nodes

def analyze_target_side(side_name, sampler, target_nodes_chains, qubits_used):
    """
    Runs the full analysis (build sets + run heuristic) for a given side.
    """
    print("\n" + "===" * 15)
    print(f"--- Analyzing Target Side: {side_name} ---")
    print("===" * 15)
    
    # Build the V_i sets
    v_sets, v_all = build_neighbor_sets(sampler, target_nodes_chains, qubits_used)
    
    if not v_all:
        print("\nNo available neighbors found for any target node on this side.")
        print(f"Estimated max conditioning nodes for {side_name}: 0")
        return 0

    # Run the heuristic
    num_nodes, node_qubit_sets = find_max_disjoint_hitting_sets_heuristic(v_sets, v_all)
    
    print("---" * 10)
    print(f"Heuristic Result for {side_name}: {num_nodes}")
    print(f"   (Estimated max number of conditioning nodes)")
    print("---" * 10)
    
    if num_nodes > 0:
        print("\nPhysical qubit set sizes for each found node:")
        print(sum([len(s) for s in node_qubit_sets]))
    
    return num_nodes, node_qubit_sets



def visualize_embedding(sampler, graph, left_chains_dict, right_chains_dict, conditioning_sets, colors):
    """
    Draws the QPU graph with all embedded nodes colored using draw_zephyr_embedding.
    """
    print("\n--- 4. Generating Visualization ---")
    
    # 1. Build the 'emb' (embedding) dictionary
    #    This maps a unique logical node ID to its physical chain (or set)
    emb = {}
    
    # Add left chains: {'L_0': [q1, q2], 'L_1': [q3, q4], ...}
    for logical_node, chain in left_chains_dict.items():
        emb[f'L_{logical_node}'] = chain
        
    # Add right chains: {'R_0': [q5, q6], 'R_1': [q7, q8], ...}
    for logical_node, chain in right_chains_dict.items():
        emb[f'R_{logical_node}'] = chain
        
    # Add conditioning nodes: {'C_0': {q9, q10}, 'C_1': {q11, q12}, ...}
    for i, q_set in enumerate(conditioning_sets):
        emb[f'C_{i}'] = q_set # The function accepts iterables (sets are fine)

    # 2. Build the 'chain_color' dictionary
    #    This maps the same unique logical IDs to their colors
    chain_color = {}
    for logical_node in emb:
        if logical_node.startswith('L_'):
            chain_color[logical_node] = colors['left']
        elif logical_node.startswith('R_'):
            chain_color[logical_node] = colors['right']
        elif logical_node.startswith('C_'):
            chain_color[logical_node] = colors['cond']

    print("Embedding and color map created. Drawing plot (this may take a moment)...")

    # 3. Draw the graph using the correct function
    plt.figure(figsize=(18, 18))
    
    dnx.draw_zephyr_embedding(
        graph, 
        # sampler=sampler,
        emb=emb,
        chain_color=chain_color,
        unused_color=colors['unused_tuple'], # Must be an RGBA tuple
        node_size=10,
        show_labels=False, # This is the correct param instead of with_labels
        width=0.5 # Edge line width
        # REMOVED: edge_color=colors['edges'] -- This caused the TypeError
    )
    
    # 4. Create a legend
    # We still create the legend manually, but we get the counts
    # for a more informative label.
    left_qubits = set()
    for chain in left_chains_dict.values():
        left_qubits.update(chain)
        
    right_qubits = set()
    for chain in right_chains_dict.values():
        right_qubits.update(chain)
        
    conditioning_qubits = set()
    for q_set in conditioning_sets:
        conditioning_qubits.update(q_set)
        
    # We create dummy plots for the legend handles
    l_patch = plt.Line2D([0], [0], marker='o', color='w', label=f'Left Chains ({len(left_qubits)} qubits)',
                          markerfacecolor=colors['left'], markersize=10)
    r_patch = plt.Line2D([0], [0], marker='o', color='w', label=f'Right Chains ({len(right_qubits)} qubits)',
                          markerfacecolor=colors['right'], markersize=10)
    c_patch = plt.Line2D([0], [0], marker='o', color='w', label=f'Conditioning Nodes ({len(conditioning_qubits)} qubits)',
                          markerfacecolor=colors['cond'], markersize=10)
    u_patch = plt.Line2D([0], [0], marker='o', color='w', label='Unused Qubits',
                          markerfacecolor=colors['unused'], markersize=10) # Use hex for legend
    
    plt.legend(handles=[l_patch, r_patch, c_patch, u_patch], loc='upper right', fontsize=18)
    plt.title(f"Embedding on {sampler.solver.name}", fontsize=20)
    plt.show()


In [52]:
RBM_SIZE = 76
SOLVER = "Advantage2_system1.7"

COLORS = {
    'left': '#FF5733',  # Orange-Red
    'right': '#33FF57', # Green
    'cond': '#3357FF',  # Blue
    'unused': '#E0E0E0', # Light Gray (for legend)
    'unused_tuple': (0.878, 0.878, 0.878, 1.0), # RGBA tuple for the function
    'edges': '#B0B0B0'  # Medium Gray
}

sampler, working_graph, q_used, l_chains, r_chains = run_embedding(RBM_SIZE, SOLVER)

if l_chains is None or r_chains is None:
    print("\nEmbedding failed. Exiting.")
    sys.exit(1)
    
# --- Analyze Left Chains ---
target_nodes_left = list(l_chains.values())
num_nodes_left, cond_sets_left = analyze_target_side(
    "Left Chains", sampler, target_nodes_left, q_used
)

# --- Analyze Right Chains ---
target_nodes_right = list(r_chains.values())
num_nodes_right, cond_sets_right = analyze_target_side(
    "Right Chains", sampler, target_nodes_right, q_used
)

# --- Final Summary ---
print("\n" + "===" * 15)
print("--- Overall Summary ---")
print(f"Max nodes if connecting to Left Chains:  {num_nodes_left}")
print(f"Max nodes if connecting to Right Chains: {num_nodes_right}")

best_cond_sets = []
if num_nodes_left >= num_nodes_right:
    print("\n The 'Left Chains' side appears to be the better choice.")
    best_cond_sets = cond_sets_left
else:
    print("\nThe 'Right Chains' side appears to be the better choice.")
    best_cond_sets = cond_sets_right
print("===" * 15)

# --- Visualize the Result ---
if working_graph:
    visualize_embedding(
        sampler,
        working_graph,
        l_chains,
        r_chains,
        best_cond_sets,
        COLORS
    )
else:
    print("\nCannot visualize: working_graph is not defined.")
    

In [11]:
# Find the embedding of a 76x76 bipartite RBM onto the D-Wave Advantage2_system1.7 QPU
rbm_size = 76
TARGET_SOLVER = "Advantage2_system1.7"
target_sampler = DWaveSampler(solver=TARGET_SOLVER)
working_graph = target_sampler.to_networkx_graph()

left_chains, right_chains = find_biclique_embedding(
                rbm_size, rbm_size, target_graph=working_graph
            )
all_chains = list(left_chains.values()) + list(right_chains.values())
max_len = max(len(chain) for chain in all_chains)
num_qubits_used = sum(len(chain) for chain in all_chains)

# This is the set of ALL qubits used in the 76x76 embedding
qubits_used = set()
for chain in all_chains:
    qubits_used.update(chain)

print(f" -> Max chain length: {max_len}")
print(f" -> Total qubits used: {len(qubits_used)}")
max_practical_a = rbm_size
# plot distribution of chain lengths
chain_lengths = [len(chain) for chain in all_chains]
plt.hist(chain_lengths, bins=range(1, max_len + 2), align='left', rwidth=0.8)
plt.title(f"Chain Length Distribution for K_{rbm_size},{rbm_size} on {TARGET_SOLVER}")
plt.xlabel("Chain Length")
plt.ylabel("Frequency")
plt.show()

print("\n--- Calculating Max Conditioning Nodes ---")

# 1. Get all qubits available on the chip
all_physical_qubits = set(target_sampler.nodelist)
qubits_avail = all_physical_qubits - qubits_used
print(f"Total available qubits: {len(qubits_avail)}")

# 2. Get the adjacency property of the sampler
qpu_adj = target_sampler.adjacency

# 3. This is the side (A) we want to connect to.
#    We assume it's 'left_chains'.
target_logical_nodes = left_chains.values() 

min_adj_size = float('inf')
bottleneck_node_chain = None

# 4. Iterate over each of the 76 logical nodes in Side A
for i, chain in enumerate(target_logical_nodes):
    
    # This set will hold all *available* neighbors for this one logical node
    chain_available_neighbors = set()
    
    # 5. Check neighbors for every physical qubit in the chain
    for q_in_chain in chain:
        for neighbor in qpu_adj[q_in_chain]:
            # 6. If the neighbor is in the available set, add it
            if neighbor in qubits_avail:
                chain_available_neighbors.add(neighbor)
    
    current_adj_size = len(chain_available_neighbors)
    # print(f"Logical node {i} (chain len {len(chain)}) has {current_adj_size} available neighbors.")

    # 7. Check if this is our new bottleneck
    if current_adj_size < min_adj_size:
        min_adj_size = current_adj_size
        bottleneck_node_chain = chain

print("---" * 10)
print(f"The bottleneck is a logical node with {min_adj_size} available neighbors.")
if bottleneck_node_chain:
    print(f"(Chain: {bottleneck_node_chain})")

print(f"\n✅ Maximum number of conditioning nodes you can add: {min_adj_size}")
print("---" * 10)

In [ ]:
#!/usr/bin/env python3


# --- Configuration ---
# The logical graph we want to embed
BIPARTITE_N = 184
# The specific D-Wave Advantage2 solver to target
# As of May 2025, this is the GA Advantage2 solver 
TARGET_SOLVER = "Advantage2_system1.7" 

print(f"--- STARTING EMBEDDING TEST ---")
print(f"Attempting to embed a K_{BIPARTITE_N},{BIPARTITE_N} graph.")
print(f"Targeting D-Wave solver: {TARGET_SOLVER}\n")

# ------------------------------------------------------------------
# STEP 1: FETCH THE TARGET HARDWARE GRAPH (THE 'WORKING GRAPH')
# ------------------------------------------------------------------
try:
    # Instantiate the DWaveSampler for the specific solver 
    # This requires your API key to be configured.
    sampler = DWaveSampler(solver=TARGET_SOLVER)
    
    # Get the target graph structure.
    # The 'solver.structure' attribute contains the 'nodelist', 'edgelist', 
    # and 'adjacency' of the QPU's working graph.
    # We only need the edgelist for minorminer.
    print(f"Successfully connected to {TARGET_SOLVER}.")
    print(f"Fetching hardware 'working graph'...")
    
    # This is the edgelist of the *actual* hardware
    target_edgelist = sampler.structure.edgelist
    
    # For context, let's see how many qubits/couplers are active.
    # This will NOT be the theoretical 4,800 qubits.
    print(f" -> Fetched working graph with {len(sampler.structure.nodelist)} active qubits.")
    print(f" -> Fetched working graph with {len(target_edgelist)} active couplers.\n")

except Exception as e:
    print(f"Error: Could not connect to D-Wave solver '{TARGET_SOLVER}'.")
    print(f"Please check your API key and solver name.")
    print(f"Full error: {e}")
    sys.exit(1)

# ------------------------------------------------------------------
# STEP 2: DEFINE THE LOGICAL SOURCE GRAPH
# ------------------------------------------------------------------
print(f"Creating logical source graph: K_{BIPARTITE_N},{BIPARTITE_N}...")

# Create the complete bipartite graph K_184,184 using networkx
# 
G_source = nx.complete_bipartite_graph(BIPARTITE_N, BIPARTITE_N)

# The minorminer 'find_embedding' function takes an edgelist as input
# for the source graph [14]
source_edgelist = list(G_source.edges())

print(f" -> Source graph has {G_source.number_of_nodes()} nodes.")
print(f" -> Source graph has {G_source.number_of_edges()} edges.\n")


# ------------------------------------------------------------------
# STEP 3 & 4: RUN THE EMBEDDING HEURISTIC & ANALYZE RESULT
# ------------------------------------------------------------------
print(f"Attempting to find embedding... (This may take some time)")

# Call the minorminer heuristic 
# S = source graph (edgelist)
# T = target graph (edgelist)
try:
    embedding = find_embedding(source_edgelist, target_edgelist)
    
    print("\n--- EMBEDDING TEST COMPLETE ---")
    
    # CRITICAL: How to check for failure.
    # The minorminer heuristic returns an EMPTY DICTIONARY
    # if it fails to find an embedding.[14]
    if not embedding:
        print("Result: FAILED")
        print(f"minorminer.find_embedding returned an empty dictionary.")
        print(f"A K_{BIPARTITE_N},{BIPARTITE_N} graph could NOT be embedded onto {TARGET_SOLVER}.")
    else:
        print("Result: SUCCESS")
        print(f"A K_{BIPARTITE_N},{BIPARTITE_N} graph WAS successfully embedded!")
        print("\nEmbedding Details:")
        
        # Calculate and display max chain length
        chain_lengths = [len(chain) for chain in embedding.values()]
        max_chain_len = max(chain_lengths)
        avg_chain_len = sum(chain_lengths) / len(chain_lengths)
        
        print(f" -> Maximum chain length: {max_chain_len}")
        print(f" -> Average chain length: {avg_chain_len:.2f}")

except Exception as e:
    print(f"\nAn error occurred during the embedding process: {e}")

print("--- END OF SCRIPT ---")

In [32]:
# --- Configuration ---
STARTING_A = 76  # Start from the theoretical maximum (K_184,184)
ENDING_A = 70    # Stop searching if we can't even embed this
TARGET_SOLVER = "Advantage2_system1.9" 

print("--- STARTING ITERATIVE EMBEDDING SEARCH ---")
print(f"Targeting D-Wave solver: {TARGET_SOLVER}\n")

# ------------------------------------------------------------------
# STEP 1: FETCH THE TARGET HARDWARE GRAPH (ONCE)
# ------------------------------------------------------------------
try:
    sampler = DWaveSampler(solver=TARGET_SOLVER)
    target_edgelist = sampler.structure.edgelist
    print(f"Successfully connected to {TARGET_SOLVER}.")
    print(f" -> Fetched working graph with {len(sampler.structure.nodelist)} active qubits.")
    print(f" -> Fetched working graph with {len(target_edgelist)} active couplers.\n")
except Exception as e:
    print(f"Error connecting to D-Wave solver: {e}")

# ------------------------------------------------------------------
# STEP 2: ITERATIVELY SEARCH FOR MAX K_a,a
# ------------------------------------------------------------------
max_embedded_a = None

for a in range(STARTING_A, ENDING_A - 1, -1):
    print(f"----- TESTING K_{a},{a} -----")
    
    # 1. Create the source graph
    G_source = nx.complete_bipartite_graph(a, a)
    source_edgelist = list(G_source.edges())
    print(f" -> Source graph has {G_source.number_of_nodes()} nodes.")
    
    # 2. Run the embedding heuristic
    try:
        embedding = find_embedding(source_edgelist, target_edgelist)
        
        # 3. Check for success
        if not embedding:
            print(f" -> Result: FAILED (minorminer returned empty dict)")
        else:
            print(f" -> Result: SUCCESS!")
            print(f"Found a valid embedding for K_{a},{a}.")
            
            chain_lengths = [len(chain) for chain in embedding.values()]
            print(f" -> Max chain length: {max(chain_lengths)}")
            
            max_embedded_a = a
            break # We found the max, so we can stop searching

    except Exception as e:
        print(f" -> An error occurred during embedding for K_{a},{a}: {e}")

# ------------------------------------------------------------------
# FINAL REPORT
# ------------------------------------------------------------------
print("\n--- ITERATIVE SEARCH COMPLETE ---")
if max_embedded_a:
    print(f"The largest complete bipartite graph K_a,a found was:")
    print(f"    K_{max_embedded_a},{max_embedded_a}")
else:
    print(f"No successful embedding found for any K_a,a from")
    print(f"a = {STARTING_A} down to a = {ENDING_A}.")

print("--- END OF SCRIPT ---")


In [34]:
# --- Configuration ---
# The K_a,a graph to embed
A_B = 184
# The Zephyr grid parameter for Advantage2
M_ZEPHYR = 12
# The live solver to target (as you specified)
TARGET_SOLVER = "Advantage2_system1.9" 

print("--- STARTING ZEPHYR BICLIQUE EMBEDDING TEST ---")
print(f"Targeting graph: K_{A_B},{A_B}")
print(f"Zephyr grid parameter: m = {M_ZEPHYR}\n")

# ------------------------------------------------------------------
# TEST 1: THEORETICAL (FULL-YIELD) EMBEDDING
# ------------------------------------------------------------------
print("--- TEST 1: THEORETICAL (FULL-YIELD) EMBEDDING ---")
print(f"Attempting to embed K_{A_B},{A_B} on a *perfect* Z_{M_ZEPHYR} graph...")

try:
    # We call the function using the 'm' parameter.
    # This *generates* a perfect Z_12 graph in memory.
    # We do not provide a 'target_graph'.
    left_chains_theory, right_chains_theory = find_biclique_embedding(
        A_B, A_B, m=M_ZEPHYR
    )
    
    print("\nResult: SUCCESS (As Expected)")
    print(f"Successfully found an embedding for K_{A_B},{A_B} on a perfect Z_{M_ZEPHYR} graph.")
    
    # Verify chain lengths
    all_chains_theory = list(left_chains_theory.values()) + \
                        list(right_chains_theory.values())
    max_chain_len = max(len(chain) for chain in all_chains_theory)
    
    print(f" -> Total logical variables: {len(all_chains_theory)}")
    print(f" -> Max chain length: {max_chain_len}")
    
    if max_chain_len == M_ZEPHYR:
        print(f" -> Chain length of {max_chain_len} matches the theoretical paper.[1]")

except Exception as e:
    print(f"\nResult: FAILED (Unexpected)")
    print(f"An error occurred during theoretical embedding: {e}")


# ------------------------------------------------------------------
# TEST 2: PRACTICAL (WORKING-GRAPH) EMBEDDING @ K_184,184
# ------------------------------------------------------------------
print("\n--- TEST 2: PRACTICAL (WORKING-GRAPH) EMBEDDING ---")
print(f"Attempting to embed K_{A_B},{A_B} on the *live* {TARGET_SOLVER}...")

# This variable will hold the fetched graph for Test 3
working_graph = None 

try:
    # 1. Connect to the sampler
    print(f"Connecting to {TARGET_SOLVER}...")
    sampler = DWaveSampler(solver=TARGET_SOLVER)
    
    # 2. Fetch the *actual* working graph as a networkx object
    print("Fetching QPU 'working graph'...")
    working_graph = sampler.to_networkx_graph()
    
    # This will show ~4400+, NOT the theoretical 4,800 [2, 3, 4]
    print(f" -> Working graph has {working_graph.number_of_nodes()} active qubits.")
    print(f" -> Working graph has {working_graph.number_of_edges()} active couplers.")

    # 3. Attempt embedding on the 'working_graph'
    # NOTE: We pass 'target_graph=working_graph' and 'm=None' (default)
    print(f"Attempting embedding on {working_graph.number_of_nodes()}-qubit graph...")
    
    left_chains_practical, right_chains_practical = find_biclique_embedding(
        A_B, A_B, target_graph=working_graph
    )
    
    print("\nResult: SUCCESS (Highly Unlikely)")
    print("A valid embedding was found on the practical working graph!")
    
except Exception as e:
    print("\nResult: FAILED (As Expected)")
    print("The embedding function raised an error, indicating a valid embedding")
    print("for this 'tight-packing' problem could not be found on the defective graph.")
    print(f"\nFull Error Message:\n{e}")


# ------------------------------------------------------------------
# TEST 3: ITERATIVE SEARCH FOR *ACTUAL* MAX BICLIQUE
# ------------------------------------------------------------------
print("\n--- TEST 3: ITERATIVE PRACTICAL EMBEDDING SEARCH ---")

# We can only run this test if we successfully fetched the graph in Test 2
if working_graph is not None:
    
    # We start from one less than the theoretical max, since we know 184 fails.
    START_A = 77 
    # Let's define a reasonable stopping point
    END_A = 50 
    
    print(f"Searching for largest K_a,a on {TARGET_SOLVER} working graph...")
    print(f"Range: a = {START_A} down to {END_A}\n")
    
    max_practical_a = None
    
    for a in range(START_A, END_A - 1, -1):
        print(f"--- Attempting K_{a},{a} ---")
        try:
            # Attempt the embedding with the current 'a'
            left_chains, right_chains = find_biclique_embedding(
                a, a, target_graph=working_graph
            )
            
            # If the line above does NOT throw an error, we succeeded!
            print(f"\n>>> Result: SUCCESS!")
            print(f">>> Found embedding for K_{a},{a} on {TARGET_SOLVER}")
            
            all_chains = list(left_chains.values()) + list(right_chains.values())
            max_len = max(len(chain) for chain in all_chains)
            num_qubits_used = sum(len(chain) for chain in all_chains)
            
            print(f" -> Max chain length: {max_len}")
            print(f" -> Total qubits used: {num_qubits_used}")
            
            max_practical_a = a
            # plot distribution of chain lengths
            chain_lengths = [len(chain) for chain in all_chains]
            plt.hist(chain_lengths, bins=range(1, max_len + 2), align='left', rwidth=0.8)
            plt.title(f"Chain Length Distribution for K_{a},{a} on {TARGET_SOLVER}")
            plt.xlabel("Chain Length")
            plt.ylabel("Frequency")
            plt.show()
            break # Exit the loop, we found the largest 'a'
            
        except Exception as e:
            # This is the expected path for 'a' values that are too large
            print(f" -> Result: FAILED")
            print(f" -> Error: {e}\n")
            
    # Final report
    print("\n--- Iterative Search Complete ---")
    if max_practical_a:
        print(f"The largest embeddable K_a,a found was: K_{max_practical_a},{max_practical_a}")
    else:
        print(f"No successful embedding found in the range a = {START_A} to {END_A}.")
        
else:
    print("Skipping Test 3 because the working graph was not fetched in Test 2.")

print("\n--- END OF SCRIPT ---")

In [ ]:
# --- Configuration ---
# The K_a,a graph to embed
A_B = 184
# The Zephyr grid parameter for Advantage2
M_ZEPHYR = 12
# The live solver to target (as you specified)
TARGET_SOLVER = "Advantage2_system1.7" 

print("--- STARTING ZEPHYR BICLIQUE EMBEDDING TEST ---")
print(f"Targeting graph: K_{A_B},{A_B}")
print(f"Zephyr grid parameter: m = {M_ZEPHYR}\n")

print(f"Attempting to embed K_{A_B},{A_B} on the *live* {TARGET_SOLVER}...")

# This variable will hold the fetched graph for Test 3
working_graph = None 

try:
    # 1. Connect to the sampler
    print(f"Connecting to {TARGET_SOLVER}...")
    sampler = DWaveSampler(solver=TARGET_SOLVER)
    
    # 2. Fetch the *actual* working graph as a networkx object
    print("Fetching QPU 'working graph'...")
    working_graph = sampler.to_networkx_graph()
    
    # This will show ~4400+, NOT the theoretical 4,800 [2, 3, 4]
    print(f" -> Working graph has {working_graph.number_of_nodes()} active qubits.")
    print(f" -> Working graph has {working_graph.number_of_edges()} active couplers.")

    # 3. Attempt embedding on the 'working_graph'
    # NOTE: We pass 'target_graph=working_graph' and 'm=None' (default)
    print(f"Attempting embedding on {working_graph.number_of_nodes()}-qubit graph...")
    
    left_chains_practical, right_chains_practical = find_biclique_embedding(
        A_B, A_B, target_graph=working_graph
    )
    
    print("\nResult: SUCCESS (Highly Unlikely)")
    print("A valid embedding was found on the practical working graph!")
    
except Exception as e:
    print("\nResult: FAILED (As Expected)")
    print("The embedding function raised an error, indicating a valid embedding")
    print("for this 'tight-packing' problem could not be found on the defective graph.")
    print(f"\nFull Error Message:\n{e}")


# ------------------------------------------------------------------
# TEST 3: ITERATIVE SEARCH FOR *ACTUAL* MAX BICLIQUE
# ------------------------------------------------------------------
print("\n--- TEST 3: ITERATIVE PRACTICAL EMBEDDING SEARCH ---")

# We can only run this test if we successfully fetched the graph in Test 2
if working_graph is not None:
    
    # We start from one less than the theoretical max, since we know 184 fails.
    START_A = 100
    # Let's define a reasonable stopping point
    END_A = 50
    
    print(f"Searching for largest K_a,a on {TARGET_SOLVER} working graph...")
    print(f"Range: a = {START_A} down to {END_A}\n")
    
    max_practical_a = None

    for a in range(START_A, END_A - 1, -1):
        print(f"--- Attempting K_{a},{a} ---")
        try:
            # Attempt the embedding with the current 'a'
            left_chains, right_chains = find_biclique_embedding(
                a, a, target_graph=working_graph
            )
            
            # If the line above does NOT throw an error, we succeeded!
            print(f"\n>>> Result: SUCCESS!")
            print(f">>> Found embedding for K_{a},{a} on {TARGET_SOLVER}")
            
            all_chains = list(left_chains.values()) + list(right_chains.values())
            max_len = max(len(chain) for chain in all_chains)
            
            print(f" -> Max chain length: {max_len}")
            
            max_practical_a = a
            break # Exit the loop, we found the largest 'a'
            
        except Exception as e:
            # This is the expected path for 'a' values that are too large
            print(f" -> Result: FAILED")
            print(f" -> Error: {e}\n")
            
    # Final report
    print("\n--- Iterative Search Complete ---")
    if max_practical_a:
        print(f"The largest embeddable K_a,a found was: K_{max_practical_a},{max_practical_a}")
    else:
        print(f"No successful embedding found in the range a = {START_A} to {END_A}.")
        
else:
    print("Skipping Test 3 because the working graph was not fetched in Test 2.")

print("\n--- END OF SCRIPT ---")

In [ ]:
# --- set the size you actually want ---
N = 200   # number of nodes PER layer (total logical nodes = 2*N)

# create node labels (you can choose any hashable labels)
L1_nodes = [f"L1_{i}" for i in range(N)]
L2_nodes = [f"L2_{i}" for i in range(N)]

G_logical = nx.Graph()
G_logical.add_nodes_from(L1_nodes, bipartite=0)
G_logical.add_nodes_from(L2_nodes, bipartite=1)

# Alternating connectivity (interpretation of your bullets):
# - even-indexed L1 nodes (i%2==0) connect to odd-indexed L2 nodes (j%2==1)
# - odd-indexed L2 nodes (i%2==1) connect to even-indexed L1 nodes (j%2==0)
for i in range(N):
    if i % 2 == 0:
        l1 = L1_nodes[i]
        for j in range(N):
            if j % 2 == 1:
                l2 = L2_nodes[j]
                G_logical.add_edge(l1, l2)
    if i % 2 == 1:
        l2 = L2_nodes[i]
        for j in range(N):
            if j % 2 == 0:
                l1 = L1_nodes[j]
                G_logical.add_edge(l2, l1)

print(f"Logical graph: {G_logical.number_of_nodes()} nodes, {G_logical.number_of_edges()} edges")

# --- get target (hardware) graph ---
# Option A: prototype locally with a perfect Zephyr of size m
G_target = dnx.zephyr_graph(16)   # choose m that gives enough qubits (increase m if needed)
print(f"Target zephyr_graph (m=16): {G_target.number_of_nodes()} nodes, {G_target.number_of_edges()} edges")

# Option B: (preferred if you will run on a real sampler)
# sampler = DWaveSampler()                 # uncomment if you have credentials
# G_target = sampler.to_networkx_graph()   # real hardware topology (with defects accounted for)

# --- try minorminer ---
params = dict(tries=30, timeout=60, threads=4, chainlength_patience=20, verbose=2)
embedding = minorminer.find_embedding(G_logical, G_target, **params)

if embedding:
    chain_lengths = [len(chain) for chain in embedding.values()]
    print("Embedding found")
    print(f"  max chain length: {max(chain_lengths)}")
    print(f"  avg chain length: {sum(chain_lengths)/len(chain_lengths):.2f}")
else:
    print("No embedding found (embedding is empty). Try reducing density or increasing tries/target size.")


In [ ]:
import networkx as nx
import dwave_networkx as dnx
import minorminer
# from dwave.system import DWaveSampler # Uncomment for real hardware

# --- set the size you actually want ---
N = 100     # number of nodes PER layer (total logical nodes = 2*N)
K_NEIGHBORS = 4  # The number of neighbors for each L1 node.
               # Must be >= 2 to ensure a connected graph.

def create_logical_graph(n_nodes, k_neighbors):
    """
    Creates a bipartite graph with n_nodes in each layer (L1, L2).
    
    Connects each node L1[i] to k_neighbors nodes in L2, starting
    from L2[i] and wrapping around the layer using modulo.
    
    This creates a single, connected graph (for k_neighbors >= 2) 
    with a total of (n_nodes * k_neighbors) edges.
    """
    L1_nodes = [f"L1_{i}" for i in range(n_nodes)]
    L2_nodes = [f"L2_{i}" for i in range(n_nodes)]

    G_logical = nx.Graph()
    G_logical.add_nodes_from(L1_nodes, bipartite=0)
    G_logical.add_nodes_from(L2_nodes, bipartite=1)

    print(f"Building logical graph with N={n_nodes} and K_NEIGHBORS={k_neighbors}...")
    
    if k_neighbors > n_nodes:
        print(f"Warning: k_neighbors ({k_neighbors}) is > N ({n_nodes}). Clamping to N.")
        k_neighbors = n_nodes
    elif k_neighbors < 2:
        print(f"Warning: k_neighbors ({k_neighbors}) < 2. The resulting graph will not be connected.")

    # This is the generalized "k-neighbor" logic:
    for i in range(n_nodes):
        l1 = L1_nodes[i]
        for offset in range(k_neighbors):
            # Connect L1[i] to L2[j], L2[j+1], ...
            j = (i + offset) % n_nodes
            l2 = L2_nodes[j]
            G_logical.add_edge(l1, l2)
                
    return G_logical

# --- create the logical graph ---
G_logical = create_logical_graph(N, K_NEIGHBORS)
print(f"Logical graph (N={N}, K={K_NEIGHBORS}): {G_logical.number_of_nodes()} nodes, {G_logical.number_of_edges()} edges")
print(f"Graph is connected: {nx.is_connected(G_logical)}")

# --- create the logical graph for visualization ---
G_logical_viz = create_logical_graph(N, K_NEIGHBORS)
print(f"Logical graph for visualization (N={N}, K={K_NEIGHBORS}): {G_logical_viz.number_of_nodes()} nodes, {G_logical_viz.number_of_edges()} edges")
print(f"Graph is connected: {nx.is_connected(G_logical_viz)}")


# --- Visualize the graph ---
plt.figure(figsize=(10, 8))

# Define node sets for bipartite layout
l1_nodes_viz = [f"L1_{i}" for i in range(N)]
l2_nodes_viz = [f"L2_{i}" for i in range(N)]

# Create bipartite layout
pos = nx.bipartite_layout(G_logical_viz, l1_nodes_viz)

# Draw nodes
nx.draw_networkx_nodes(G_logical_viz, pos, nodelist=l1_nodes_viz, node_color='skyblue', label='Layer 1')
nx.draw_networkx_nodes(G_logical_viz, pos, nodelist=l2_nodes_viz, node_color='lightcoral', label='Layer 2')

# Draw edges
nx.draw_networkx_edges(G_logical_viz, pos, edge_color='gray', alpha=0.7)

# Draw labels
nx.draw_networkx_labels(G_logical_viz, pos, font_size=8)

plt.title(f"Custom Bipartite Graph (N={N} per layer, K_NEIGHBORS={K_NEIGHBORS})")
plt.legend()
plt.axis('off') # Hide axes
plt.show()


In [ ]:
# --- get target (hardware) graph ---
# Option A: prototype locally with a perfect Zephyr of size m
G_target = dnx.zephyr_graph(16)   # choose m that gives enough qubits (increase m if needed)
print(f"Target zephyr_graph (m=16): {G_target.number_of_nodes()} nodes, {G_target.number_of_edges()} edges")

# Option B: (preferred if you will run on a real sampler)
# sampler = DWaveSampler()                 # uncomment if you have credentials
# G_target = sampler.to_networkx_graph()   # real hardware topology (with defects accounted for)
# print(f"Target hardware graph: {G_target.number_of_nodes()} nodes, {G_target.number_of_edges()} edges")


# --- try minorminer ---
print("\nStarting minorminer.find_embedding...")
params = dict(tries=30, timeout=60, threads=4, chainlength_patience=20, verbose=2)
embedding = minorminer.find_embedding(G_logical, G_target, **params)

if embedding:
    chain_lengths = [len(chain) for chain in embedding.values()]
    print("\nEmbedding found!")
    print(f"  Max chain length: {max(chain_lengths)}")
    print(f"  Avg chain length: {sum(chain_lengths)/len(chain_lengths):.2f}")
else:
    print("\nNo embedding found (embedding is empty).")
    print("Try increasing K (to make the graph sparser), increasing 'tries'/'timeout', or using a larger target graph (increase 'm').")


In [ ]:
def create_sparse_connected_bipartite(n: int, target_edges: int):
    """
    Generates a sparse, connected, random bipartite graph of size n x n.

    This function implements a "build-up" method:
    1. It starts with two partitions of n nodes each (2n nodes total).
    2. It adds random bipartite edges until the graph is connected.
    3. It continues adding random bipartite edges until target_edges is reached.

    Args:
        n: The number of nodes in each of the two partitions.
        target_edges: The total number of edges the final graph should have.

    Returns:
        A networkx Graph object.

    Raises:
        ValueError: If the target_edges is not possible.
            - If target_edges is less than the minimum (2n - 1) required
              for a 2n-node graph to be connected.
            - If n is 0 or negative.
    """
    if n <= 0:
        raise ValueError("n must be a positive integer.")

    # --- 1. Validation ---
    min_edges = 2 * n - 1  # Minimum edges for a connected graph (spanning tree)
    max_edges = n * n      # Maximum edges in a complete K_n,n graph

    if target_edges < min_edges:
        raise ValueError(
            f"Target edges ({target_edges}) is less than the minimum ({min_edges}) "
            f"required for a connected graph of {2*n} nodes."
        )

    if target_edges > max_edges:
        print(
            f"Warning: Target edges ({target_edges}) exceeds the maximum possible "
            f"({max_edges}). Returning a complete bipartite K_{n},{n} graph."
        )
        target_edges = max_edges

    # --- 2. Initialization ---
    G = nx.Graph()
    partition_U = list(range(n))
    partition_V = list(range(n, 2 * n))
    
    # Add nodes with bipartite attribute
    G.add_nodes_from(partition_U, bipartite=0)
    G.add_nodes_from(partition_V, bipartite=1)

    # Generate all possible bipartite edges
    possible_edges = [
        (u, v) for u in partition_U for v in partition_V
    ]
    random.shuffle(possible_edges)
    
    edge_iterator = iter(possible_edges)

    # --- 3. Phase 1: Ensure Connectivity (Build Spanning Tree) ---
    # We add edges until the graph has only one connected component
    while not nx.is_connected(G):
        try:
            u, v = next(edge_iterator)
            G.add_edge(u, v)
        except StopIteration:
            # This should only happen if n=1 and target_edges=1
            break 
            
    # --- 4. Phase 2: Add Remaining Edges to Reach Target ---
    num_edges_to_add = target_edges - G.number_of_edges()

    for _ in range(num_edges_to_add):
        try:
            u, v = next(edge_iterator)
            G.add_edge(u, v)
        except StopIteration:
            # This means we've added all possible n*n edges
            break
            
    return G, partition_U, partition_V

N_NODES = 200 
TARGET_EDGES = 50 * N_NODES

print(f"Generating a {N_NODES}x{N_NODES} sparse bipartite graph...")
print(f"Target edges: {TARGET_EDGES}\n")

try:
    G, U, V = create_sparse_connected_bipartite(N_NODES, TARGET_EDGES)

    print("--- Graph Generation Successful ---")
    print(f"Partition U nodes: {U}")
    print(f"Partition V nodes: {V}")
    print(f"Total nodes: {G.number_of_nodes()}")
    print(f"Final edges: {G.number_of_edges()}")
    print(f"Is connected: {nx.is_connected(G)}")
    print(f"Is bipartite: {nx.is_bipartite(G)}")

    # Example of low target edge count
    print("\n--- Testing minimum connectivity ---")
    min_edges_required = 2 * N_NODES - 1
    G_min, _, _ = create_sparse_connected_bipartite(N_NODES, min_edges_required)
    print(f"Targeting minimum edges: {min_edges_required}")
    print(f"Resulting edges: {G_min.number_of_edges()}")
    print(f"Is connected: {nx.is_connected(G_min)}")

except ValueError as e:
    print(f"Error generating graph: {e}")

#plot generated graph
plt.figure(figsize=(8, 6))
pos = nx.bipartite_layout(G, U)
nx.draw(G, pos, with_labels=True, node_color=['lightblue' if n in U else 'lightgreen' for n in G.nodes()])
plt.title(f"Sparse Bipartite Graph with {G.number_of_edges()} Edges")
plt.show()


In [ ]:
# --- get target (hardware) graph ---
# Option A: prototype locally with a perfect Zephyr of size m
G_target = dnx.zephyr_graph(16)   # choose m that gives enough qubits (increase m if needed)
print(f"Target zephyr_graph (m=16): {G_target.number_of_nodes()} nodes, {G_target.number_of_edges()} edges")

# Option B: (preferred if you will run on a real sampler)
# sampler = DWaveSampler()                 # uncomment if you have credentials
# G_target = sampler.to_networkx_graph()   # real hardware topology (with defects accounted for)
# print(f"Target hardware graph: {G_target.number_of_nodes()} nodes, {G_target.number_of_edges()} edges")


# --- try minorminer ---
print("\nStarting minorminer.find_embedding...")
params = dict(tries=30, timeout=60, threads=4, chainlength_patience=20, verbose=2)
embedding = minorminer.find_embedding(G, G_target, **params)

if embedding:
    chain_lengths = [len(chain) for chain in embedding.values()]
    print("\nEmbedding found!")
    print(f"  Max chain length: {max(chain_lengths)}")
    print(f"  Avg chain length: {sum(chain_lengths)/len(chain_lengths):.2f}")
else:
    print("\nNo embedding found (embedding is empty).")
    print("Try increasing K (to make the graph sparser), increasing 'tries'/'timeout', or using a larger target graph (increase 'm').")
